# Let's try to redo the project with data from the past
---
After looking at the the past data, we will have to rescrape new data.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pybaseball as pb
from pybaseball import cache

pd.set_option('display.max_columns', None)

In [2]:
# mlb2020 = pb.statcast(start_dt='2020-07-23', end_dt='2020-10-27')
# mlb2019 = pb.statcast(start_dt='2019-03-20', end_dt='2019-10-30')
# mlb2018 = pb.statcast(start_dt='2018-03-29', end_dt='2018-10-28')
# mlb2017 = pb.statcast(start_dt='2017-04-02', end_dt='2017-11-01')
# mlb2016 = pb.statcast(start_dt='2016-04-03', end_dt='2016-11-02')
# mlb2015 = pb.statcast(start_dt='2015-04-05', end_dt='2015-11-01')
print('Retrieval Complete')

This is a large query, it may take a moment to complete
Completed sub-query from 2020-07-23 to 2020-07-28
Completed sub-query from 2020-07-29 to 2020-08-03
Completed sub-query from 2020-08-04 to 2020-08-09
Completed sub-query from 2020-08-10 to 2020-08-15
Completed sub-query from 2020-08-16 to 2020-08-21
Completed sub-query from 2020-08-22 to 2020-08-27
Completed sub-query from 2020-08-28 to 2020-09-02
Completed sub-query from 2020-09-03 to 2020-09-08
Completed sub-query from 2020-09-09 to 2020-09-14
Completed sub-query from 2020-09-15 to 2020-09-20
Completed sub-query from 2020-09-21 to 2020-09-26
Completed sub-query from 2020-09-27 to 2020-10-02
Completed sub-query from 2020-10-03 to 2020-10-08
Completed sub-query from 2020-10-09 to 2020-10-14
Completed sub-query from 2020-10-15 to 2020-10-20
Completed sub-query from 2020-10-21 to 2020-10-26
Completed sub-query from 2020-10-27 to 2020-10-27
This is a large query, it may take a moment to complete
Completed sub-query from 2019-03-20 to

In [4]:
# mlb2020.to_csv('rescrape-past-seasons/2020mlb.csv', index=False)
# mlb2019.to_csv('rescrape-past-seasons/2019mlb.csv', index=False)
# mlb2018.to_csv('rescrape-past-seasons/2018mlb.csv', index=False)
# mlb2017.to_csv('rescrape-past-seasons/2017mlb.csv', index=False)
# mlb2016.to_csv('rescrape-past-seasons/2016mlb.csv', index=False)
# mlb2015.to_csv('rescrape-past-seasons/2015mlb.csv', index=False)

We will use the same cleaning and cleaning order to get the exact things we have completed.

In [7]:
mlb2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279660 entries, 0 to 279659
Data columns (total 93 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   index                            279660 non-null  int64         
 1   pitch_type                       279636 non-null  object        
 2   game_date                        279660 non-null  datetime64[ns]
 3   release_speed                    279636 non-null  float64       
 4   release_pos_x                    279636 non-null  float64       
 5   release_pos_z                    279636 non-null  float64       
 6   player_name                      279660 non-null  object        
 7   batter                           279660 non-null  float64       
 8   pitcher                          279660 non-null  float64       
 9   events                           70400 non-null   object        
 10  description                      279660 non-

In [8]:
mlb2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712840 entries, 0 to 712839
Data columns (total 93 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   index                            712840 non-null  int64         
 1   pitch_type                       712173 non-null  object        
 2   game_date                        712840 non-null  datetime64[ns]
 3   release_speed                    712782 non-null  float64       
 4   release_pos_x                    712173 non-null  float64       
 5   release_pos_z                    712173 non-null  float64       
 6   player_name                      712840 non-null  object        
 7   batter                           712840 non-null  float64       
 8   pitcher                          712840 non-null  float64       
 9   events                           186657 non-null  object        
 10  description                      712840 non-

In [9]:
def full_clean(df):
    # Drops the columns we don't want
    df.drop(columns=['events','spin_dir','spin_rate_deprecated','break_angle_deprecated',
                     'break_length_deprecated','game_type','home_team','away_team',
                     'hit_location','game_year','on_3b','on_2b','on_1b',
                     'outs_when_up','inning','inning_topbot','hc_x','hc_y',
                     'tfs_deprecated','tfs_zulu_deprecated','umpire','sv_id',
                     'hit_distance_sc','launch_speed','launch_angle',
                     'game_pk', 'pitcher.1','fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 
                     'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9',
                     'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
                     'woba_value', 'woba_denom', 'babip_value', 'iso_value',
                     'launch_speed_angle', 'at_bat_number', 'pitch_number',
                     'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score',
                     'post_home_score', 'post_bat_score', 'post_fld_score',
                     'if_fielding_alignment', 'of_fielding_alignment', 'spin_axis',
                     'delta_home_win_exp', 'delta_run_exp'], inplace=True)
    
    # Keeps rows that didn't result in getting hit by the bat
    df = df[df['bb_type'].isnull()]
    
    # We only want data that was a called strike or called ball
    df = df.loc[((df['description'] == 'ball') | (df['description'] == 'called_strike'))]
    
    # We don't need bb_type anymore
    df.drop(columns=['bb_type'], inplace=True)
    
    # At this point, any nulls in any row is a row that isn't needed
    df.dropna(inplace=True)
    
    # We also want to return to different versions of the df for EDA and modeling purposes
    df2 = df.loc[:, ~df.columns.isin(['index','game_date','player_name','batter','pitcher','description','des',
                'fielder_2','sz_top','sz_bot'])]

    df3 = df2.loc[:, ~df2.columns.isin(['zone','balls','strikes','effective_speed','release_extension',
                                        'release_pos_y','pitch_type'])]
    
    # Return 3 variables
    return df, df2, df3
    

In [10]:
mlb2020, mlb2020_unnecessary, train_test_2020 = full_clean(mlb2020)
mlb2019, mlb2019_unnecessary, train_test_2019 = full_clean(mlb2019)
mlb2018, mlb2018_unnecessary, train_test_2018 = full_clean(mlb2018)
mlb2017, mlb2017_unnecessary, train_test_2017 = full_clean(mlb2017)
mlb2016, mlb2016_unnecessary, train_test_2016 = full_clean(mlb2016)
mlb2015, mlb2015_unnecessary, train_test_2015 = full_clean(mlb2015)

In [14]:
# Saving the data

# 2020
mlb2020.to_csv('rescrape-past-seasons/2020cleaned.csv', index=False)
mlb2020_unnecessary.to_csv('rescrape-past-seasons/2020unnecessary.csv', index=False)
train_test_2020.to_csv('train_test_2020.csv', index=False)

# 2019
mlb2019.to_csv('rescrape-past-seasons/2019cleaned.csv', index=False)
mlb2019_unnecessary.to_csv('rescrape-past-seasons/2019unnecessary.csv', index=False)
train_test_2019.to_csv('train_test_2019.csv', index=False)

# 2018
mlb2018.to_csv('rescrape-past-seasons/2018cleaned.csv', index=False)
mlb2018_unnecessary.to_csv('rescrape-past-seasons/2018unnecessary.csv', index=False)
train_test_2018.to_csv('train_test_2018.csv', index=False)

# 2017
mlb2017.to_csv('rescrape-past-seasons/2017cleaned.csv', index=False)
mlb2017_unnecessary.to_csv('rescrape-past-seasons/2017unnecessary.csv', index=False)
train_test_2017.to_csv('train_test_2017.csv', index=False)

# 2016
mlb2016.to_csv('rescrape-past-seasons/2016cleaned.csv', index=False)
mlb2016_unnecessary.to_csv('rescrape-past-seasons/2016unnecessary.csv', index=False)
train_test_2016.to_csv('train_test_2016.csv', index=False)

# 2015
mlb2015.to_csv('rescrape-past-seasons/2015cleaned.csv', index=False)
mlb2015_unnecessary.to_csv('rescrape-past-seasons/2015unnecessary.csv', index=False)
train_test_2015.to_csv('train_test_2015.csv', index=False)

In [16]:
train_test_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332086 entries, 0 to 712786
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   release_speed      332086 non-null  float64
 1   release_pos_x      332086 non-null  float64
 2   release_pos_z      332086 non-null  float64
 3   stand              332086 non-null  object 
 4   p_throws           332086 non-null  object 
 5   type               332086 non-null  object 
 6   pfx_x              332086 non-null  float64
 7   pfx_z              332086 non-null  float64
 8   plate_x            332086 non-null  float64
 9   plate_z            332086 non-null  float64
 10  vx0                332086 non-null  float64
 11  vy0                332086 non-null  float64
 12  vz0                332086 non-null  float64
 13  ax                 332086 non-null  float64
 14  ay                 332086 non-null  float64
 15  az                 332086 non-null  float64
 16  re

In [19]:
train_test_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141578 entries, 0 to 279659
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   release_speed      141578 non-null  float64
 1   release_pos_x      141578 non-null  float64
 2   release_pos_z      141578 non-null  float64
 3   stand              141578 non-null  object 
 4   p_throws           141578 non-null  object 
 5   type               141578 non-null  object 
 6   pfx_x              141578 non-null  float64
 7   pfx_z              141578 non-null  float64
 8   plate_x            141578 non-null  float64
 9   plate_z            141578 non-null  float64
 10  vx0                141578 non-null  float64
 11  vy0                141578 non-null  float64
 12  vz0                141578 non-null  float64
 13  ax                 141578 non-null  float64
 14  ay                 141578 non-null  float64
 15  az                 141578 non-null  float64
 16  re

In [20]:
# We will concatenate the past data, just like before
old_concat = pd.concat([train_test_2015, train_test_2016, train_test_2017, train_test_2018, train_test_2019, train_test_2020],axis=0)

In [21]:
old_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1899802 entries, 0 to 279659
Data columns (total 18 columns):
 #   Column             Dtype  
---  ------             -----  
 0   release_speed      float64
 1   release_pos_x      float64
 2   release_pos_z      float64
 3   stand              object 
 4   p_throws           object 
 5   type               object 
 6   pfx_x              float64
 7   pfx_z              float64
 8   plate_x            float64
 9   plate_z            float64
 10  vx0                float64
 11  vy0                float64
 12  vz0                float64
 13  ax                 float64
 14  ay                 float64
 15  az                 float64
 16  release_spin_rate  float64
 17  pitch_name         object 
dtypes: float64(14), object(4)
memory usage: 275.4+ MB


In [22]:
old_concat.describe()

,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az,release_spin_rate
count,1.899802e+06,1.899802e+06,1.899802e+06,1.899802e+06,1.899802e+06,1.899802e+06,1.899802e+06,1.899802e+06,1.899802e+06,1.899802e+06,1.899802e+06,1.899802e+06,1.899802e+06,1.899802e+06
mean,8.885563e+01,-7.527199e-01,5.882158e+00,-2.006692e-01,7.967591e-01,1.721111e-02,2.281055e+00,2.302325e+00,-1.291068e+02,-4.390857e+00,-2.495005e+00,2.642872e+01,-2.258968e+01,2.218335e+03
std,6.023921e+00,1.868532e+00,4.856489e-01,8.868429e-01,7.727484e-01,1.018230e+00,1.050048e+00,6.190259e+00,8.714240e+00,3.379831e+00,1.078642e+01,4.064442e+00,8.982179e+00,3.235584e+02
min,4.170000e+01,-6.620000e+00,-1.840000e+00,-4.150000e+00,-2.790000e+00,-9.670000e+00,-4.520000e+00,-2.203000e+01,-1.531200e+02,-1.991990e+01,-5.929009e+01,4.290000e+00,-5.407206e+01,4.130000e+02
25%,8.490000e+01,-2.090000e+00,5.620000e+00,-9.500000e-01,3.900000e-01,-7.700000e-01,1.550000e+00,-2.090000e+00,-1.356600e+02,-6.819677e+00,-1.151000e+01,2.345237e+01,-2.837819e+01,2.076000e+03
50%,9.040000e+01,-1.440000e+00,5.910000e+00,-3.300000e-01,9.800000e-01,4.000000e-02,2.230000e+00,3.840000e+00,-1.313045e+02,-4.600000e+00,-3.570495e+00,2.648891e+01,-2.103853e+01,2.243000e+03
75%,9.340000e+01,9.800000e-01,6.190000e+00,5.300000e-01,1.370000e+00,8.000000e-01,2.980000e+00,6.862404e+00,-1.233870e+02,-2.150000e+00,6.014304e+00,2.927000e+01,-1.548815e+01,2.403000e+03
max,1.057000e+02,5.450000e+00,1.054000e+01,2.930000e+00,5.390000e+00,7.990000e+00,1.358000e+01,2.515000e+01,-5.962098e+01,2.009525e+01,4.098000e+01,5.406000e+01,3.039456e+01,3.741000e+03


In [23]:
old_concat.to_csv('past_seasons_concatenated_cleaned.csv', index=False)

In [24]:
# We will import the newest season and see if we can concat to old data
train_test_2021 = pd.read_csv('train_test_2021.csv')
train_test_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357046 entries, 0 to 357045
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   release_speed      357046 non-null  float64
 1   release_pos_x      357046 non-null  float64
 2   release_pos_z      357046 non-null  float64
 3   stand              357046 non-null  object 
 4   p_throws           357046 non-null  object 
 5   type               357046 non-null  object 
 6   pfx_x              357046 non-null  float64
 7   pfx_z              357046 non-null  float64
 8   plate_x            357046 non-null  float64
 9   plate_z            357046 non-null  float64
 10  vx0                357046 non-null  float64
 11  vy0                357046 non-null  float64
 12  vz0                357046 non-null  float64
 13  ax                 357046 non-null  float64
 14  ay                 357046 non-null  float64
 15  az                 357046 non-null  float64
 16  re

In [25]:
new_concat = pd.concat([old_concat, train_test_2021], axis=0)
new_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2256848 entries, 0 to 357045
Data columns (total 18 columns):
 #   Column             Dtype  
---  ------             -----  
 0   release_speed      float64
 1   release_pos_x      float64
 2   release_pos_z      float64
 3   stand              object 
 4   p_throws           object 
 5   type               object 
 6   pfx_x              float64
 7   pfx_z              float64
 8   plate_x            float64
 9   plate_z            float64
 10  vx0                float64
 11  vy0                float64
 12  vz0                float64
 13  ax                 float64
 14  ay                 float64
 15  az                 float64
 16  release_spin_rate  float64
 17  pitch_name         object 
dtypes: float64(14), object(4)
memory usage: 327.1+ MB


In [26]:
new_concat.describe()

,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az,release_spin_rate
count,2.256848e+06,2.256848e+06,2.256848e+06,2.256848e+06,2.256848e+06,2.256848e+06,2.256848e+06,2.256848e+06,2.256848e+06,2.256848e+06,2.256848e+06,2.256848e+06,2.256848e+06,2.256848e+06
mean,8.885575e+01,-7.457551e-01,5.877782e+00,-1.856166e-01,7.755405e-01,2.280932e-02,2.284632e+00,2.277434e+00,-1.291238e+02,-4.338657e+00,-2.420046e+00,2.646569e+01,-2.268489e+01,2.221765e+03
std,6.042672e+00,1.872093e+00,4.933922e-01,8.851897e-01,7.724520e-01,1.015469e+00,1.059555e+00,6.175790e+00,8.741405e+00,3.397518e+00,1.076700e+01,4.051716e+00,9.013975e+00,3.246940e+02
min,3.110000e+01,-6.620000e+00,-1.840000e+00,-4.150000e+00,-2.790000e+00,-9.670000e+00,-4.980000e+00,-2.203000e+01,-1.531200e+02,-2.013814e+01,-5.929009e+01,2.473310e+00,-5.407206e+01,9.500000e+01
25%,8.480000e+01,-2.080000e+00,5.620000e+00,-9.400000e-01,3.500000e-01,-7.600000e-01,1.540000e+00,-2.185378e+00,-1.357549e+02,-6.774789e+00,-1.143640e+01,2.349018e+01,-2.863000e+01,2.077000e+03
50%,9.030000e+01,-1.440000e+00,5.910000e+00,-3.100000e-01,9.500000e-01,5.000000e-02,2.230000e+00,3.806767e+00,-1.312800e+02,-4.536051e+00,-3.346355e+00,2.651793e+01,-2.121078e+01,2.245000e+03
75%,9.340000e+01,1.020000e+00,6.190000e+00,5.400000e-01,1.360000e+00,8.100000e-01,3.000000e+00,6.839161e+00,-1.233400e+02,-2.086415e+00,6.070000e+00,2.933004e+01,-1.546000e+01,2.408000e+03
max,1.057000e+02,5.450000e+00,1.054000e+01,2.930000e+00,5.390000e+00,9.110000e+00,1.358000e+01,2.515000e+01,-4.252358e+01,2.009525e+01,4.098000e+01,5.406000e+01,3.039456e+01,3.741000e+03


In [27]:
new_concat.to_csv('past_7_seasons_cleaned.csv', index=False)